In [1]:
from elasticsearch import Elasticsearch, helpers
import pandas as pd
from beautifultable import BeautifulTable

In [2]:
# Connect to Elastic search
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [3]:
# Check if the connection was successful
if es.ping():
    print("Connection successful!")
else:
    print("Connection error!")

Connection successful!


c:\users\papag\appdata\local\programs\python\python36\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [4]:
# Create a pandas dataframe from the csv file 'BX-Books'
df = pd.read_csv("BX-Books.csv")

In [5]:
# If an index named "books" already exits, delete it
if es.indices.exists(index='books'):
    es.indices.delete(index='books', ignore=400)

In [6]:
# Create new index named "books"
es.indices.create(index='books', ignore=400)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'books'}

In [7]:
# Function that reads the data from the csv
def generator(df):
    for index, line in df.iterrows():
        yield {
            "_index": "books",
            "_type": "_doc",
            "_id": f"{line['isbn']}",
            "_source": line.to_dict(),
        }

In [8]:
# Upload data into elastic search
try:
    res = helpers.bulk(es, generator(df))
    print("Upload done!")
except Exception as e:
    pass

c:\users\papag\appdata\local\programs\python\python36\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


Upload done!


In [9]:
print("""
######################################################################################
          ___           _     ___                  _      ___           _          
  ___ ___| _ ) ___  ___| |__ / __| ___ __ _ _ _ __| |_   | __|_ _  __ _(_)_ _  ___ ©
 / -_)___| _ \/ _ \/ _ \ / / \__ \/ -_) _` | '_/ _| ' \  | _|| ' \/ _` | | ' \/ -_)
 \___|   |___/\___/\___/_\_\ |___/\___\__,_|_| \__|_||_| |___|_||_\__, |_|_||_\___|
                                                                  |___/            

                          __...--~~~~~-._   _.-~~~~~--...__
                        //               `V'               \\ 
                       //                 |                 \\ 
                      //__...--~~~~~~-._  |  _.-~~~~~~--...__\\ 
                     //__.....----~~~~._\ | /_.~~~~----.....__\\
                    ====================\\|//====================
                                        `---`
######################################################################################
""")

# User enters a book lemma to search
user_search = input("Enter a lemma to search: ")


######################################################################################
          ___           _     ___                  _      ___           _          
  ___ ___| _ ) ___  ___| |__ / __| ___ __ _ _ _ __| |_   | __|_ _  __ _(_)_ _  ___ ©
 / -_)___| _ \/ _ \/ _ \ / / \__ \/ -_) _` | '_/ _| ' \  | _|| ' \/ _` | | ' \/ -_)
 \___|   |___/\___/\___/_\_\ |___/\___\__,_|_| \__|_||_| |___|_||_\__, |_|_||_\___|
                                                                  |___/            

                          __...--~~~~~-._   _.-~~~~~--...__
                        //               `V'               \ 
                       //                 |                 \ 
                      //__...--~~~~~~-._  |  _.-~~~~~~--...__\ 
                     //__.....----~~~~._\ | /_.~~~~----.....__\
                    ====================\|//====================
                                        `---`
##################################################################

In [10]:
# Run a query in index 'books' of Elasticsearch
res = es.search(index='books', query = {"match": {"book_title": user_search}}, size=10000)

In [11]:
# Create a table for better visualization of our data
table = BeautifulTable(maxwidth=120)
table.column_headers = ["BOOK RESULTS", "BM25 SCORE"]
table.set_style(BeautifulTable.STYLE_RST)

c:\users\papag\appdata\local\programs\python\python36\lib\site-packages\beautifultable\utils.py:125: FutureWarning: 'BeautifulTable.column_headers' has been deprecated in 'v1.0.0' and will be removed in 'v1.2.0'. Use 'BTColumnCollection.header' instead.
  warnings.warn(message, FutureWarning)


In [12]:
# Store the results of the query in 'temp' dictionary
temp = {}
for hit in res['hits']['hits']:
    temp[hit['_id']] = [hit['_source']['book_title'], hit['_score']]

In [13]:
# Enter the data in the table
for i in temp.items():
    table.rows.append([i[1][0], i[1][1]])

In [14]:
# Print the results
print(table)

=========================================================================================================== ============
                                               BOOK RESULTS                                                  BM25 SCORE 
=========================================================================================================== ============
                                               Clara Callan                                                    12.645   
                                      Clara Mondschein's Melancholia                                           11.622   
                                              Henry and Clara                                                  11.622   
                                              Clara : A Novel                                                  11.622   
                                          Clara Bow: Runnin' Wild                                              10.752   
                                